In [2]:
# use these magic spells to update your classes methods on-the-fly as you edit them:
%reload_ext autoreload
%autoreload 2
from pprint import pprint
from IPython.core.display import display, HTML, Markdown
import ipywidgets as widgets
# %run includeme.ipynb # include a notebook from this same directory
display(HTML("<style>.container { width:100% !important; }</style>"))
import configparser

In [ ]:
# a helper function
def orm2dict(obj):
    """Sql alchemy orm object to dictionary

    as per: https://stackoverflow.com/questions/1958219/how-to-convert-sqlalchemy-row-object-to-a-python-dict
    """
    return {c.name: getattr(obj, c.name) for c in obj.__table__.columns}

In [ ]:
# TODO: set the abs path to your ini file
config_file = "/path/to/ini/file.ini"
!ls {config_file}

In [ ]:
# test reading postgresql adr from config file
cfg = configparser.ConfigParser()
cfg.read(config_file)
sql_adr = cfg["DEFAULT"]["sqlalchemy_url"]
print("database adr:",sql_adr)

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from my_backend.api_v1.model import *

In [ ]:
# check out data schemas for API input and output
pprint(User.Cout.schema())
pprint(User.Cin.schema())
pprint(AdminUser.Cin.schema())

In [ ]:
from my_backend.session import SessionBase
from my_backend.api_v1.model.base import colsFromPydantic
from my_backend.tool import loadConfig
loadConfig(config_file)

In [ ]:
# create a Session class
class Session(SessionBase):
    sqlalchemy_url = cfg["DEFAULT"]["sqlalchemy_url"]
    to_http_exc = False

In [ ]:
# add 10 users
for i in range(10):
    with Session() as ses:
        user = User(name="kikkelis"+str(i), email="kokkelis"+str(i), extradata="some extra data"+str(i))
        ses.add(user)

In [ ]:
# query users
lis = []
with Session() as ses:
    # pick certain columns as defined in 
    # pydantic model User.Rout for ses.query
    query = ses.query(*(colsFromPydantic(User, User.Rout)))
    # query.all() returns a list of User objects
    for user in query.all():
        lis.append(User.Rout(**user))
pprint(lis)

In [ ]:
uuid = lis[0].uuid
print(uuid)

In [ ]:
with Session() as ses:
    query = ses.query(User).filter(User.uuid == uuid)
    user_ = query.all()[0]
    print(user_)

In [ ]:
user_dict = {"name":"kikkesis", "email":"kokkelis2", "uuid":uuid}
with Session() as ses:
    query = ses.query(UserInterface).filter(UserInterface.uuid == uuid)
    query.update(user_dict)

In [ ]:
"""
# update with table inheritance fails miserably
user_dict = {"name":"kikkesis", "email":"kokkelis2", "uuid":uuid, "extradata":"some"}
with Session() as ses:
    query = ses.query(UserInterface).filter(UserInterface.uuid == uuid)
    query.update(user_dict)
"""

In [ ]:
# update for table inheritance fails miserably.. so we do delete & add instead
user_dict = {"name":"kikkesis", "email":"kokkelis2", "uuid":uuid, "extradata":"some"}
with Session() as ses:
    query = ses.query(User).filter(User.uuid == uuid)
    user = query.all()[0]
    ses.delete(user)
    user = User(**user_dict)
    ses.add(user)

In [ ]:
with Session() as ses:
    query = ses.query(User).filter(User.uuid == uuid)
    print(query.all())
    #query.delete()

In [ ]:
with Session() as ses:
    query = ses.query(User).filter(User.uuid == uuid)
    # query.delete() # this m-f cleans up the whole table..!  maybe due to table inheritance..?
    for user in query.all():
        ses.delete(user)

In [ ]:
# query users (again)
lis = []
with Session() as ses:
    # pick certain columns as defined in 
    # pydantic model User.Rout for ses.query
    query = ses.query(*(colsFromPydantic(User, User.Rout)))
    # query.all() returns a list of User objects
    for user in query.all():
        lis.append(User.Rout(**user))
pprint(lis)

In [ ]:
# delete all
with Session() as ses:
    query = ses.query(User)
    # query.delete() # this m-f cleans up the whole table..!  maybe due to table inheritance..?
    for user in query.all():
        ses.delete(user)

In [ ]:
# query users (again)
lis = []
with Session() as ses:
    # pick certain columns as defined in 
    # pydantic model User.Rout for ses.query
    query = ses.query(*(colsFromPydantic(User, User.Rout)))
    # query.all() returns a list of User objects
    for user in query.all():
        lis.append(User.Rout(**user))
pprint(lis)